In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def listArrivee(tempsMax,lam):
    L=[]
    t=0.0
    
    while t<=tempsMax-dt:
        t+=np.random.exponential(lam)
        L.append(t)
    
    return np.array(L)

In [11]:
def addProcess_2S(tmax,lama,l_lamd,nbService,l_nbPers):
    
    lTmps = []
    t = 0.0  # temps du derniere evenement 
    dta = None
    dtd = []
    cpt = l_nbPers
    cpt_T = []
    b_cpt = True
    dtdMin = 0
    for i in range(nbService):
        dtd.append(l_lamd[i])
        
    while True:
        
        
        for i in cpt:
            if i != 0:
                b_cpt = False
                
        if b_cpt:
            if dta is None:
                dta = np.random.exponential(lama)
            t += dta            
            cpt[0] += 1
            dta = None
        else:
            dtdMin = np.argmin(cpt)
            if dta is None:
                dta = np.random.exponential(lama)
            if dtd[dtdMin] > dta:
                t += dta
                for i in range(nbService):
                    dtd[i] -= dta
                cpt[dtdMin] += 1
                dta = None          
            else:
                t += dtd[dtdMin]
                dta -= dtd[dtdMin]
                for i in range(nbService):
                    dtd[i] -= dtd[dtdMin]
                dtd[dtdMin] = np.random.exponential(l_lamd[dtdMin])
                cpt[dtdMin] -= 1
            
        if t > tmax:
            break

        lTmps.append(t)
        cpt_T.append(cpt)

    return lTmps,cpt_T  

In [12]:
addProcess_2S(300,10,[5,4,8],3,[0,0,0])

([11.335378210881014,
  15.335378210881014,
  17.11137120526433,
  18.95946110495444,
  19.031766168340535,
  23.18870414344041,
  24.07369771405205,
  26.603741150492667,
  29.33259367681882,
  33.15189266706602,
  35.38146492117895,
  35.62502256036356,
  36.311346403302544,
  36.34616590102066,
  36.779053256539456,
  38.03045746373904,
  40.08057796066175,
  43.108415466497846,
  49.314380251354244,
  49.59565656263555,
  53.01836178209006,
  54.604745344897054,
  58.12774871896312,
  58.833021107053504,
  74.52189728770094,
  74.86486499640301,
  75.7965457514601,
  79.32420437730018,
  83.77789438824956,
  83.88620711776518,
  88.57762925654694,
  89.95902107779007,
  93.74057482015345,
  95.09071878187603,
  99.5152670249558,
  99.9317168716955,
  123.84394993386485,
  126.5049400211491,
  129.375698822991,
  133.73159675576522,
  135.0774281557753,
  137.32763145753395,
  137.9037184411921,
  146.44451816000384,
  149.50846435973068,
  149.88942453913586,
  150.89390097033044,
